<a href="https://colab.research.google.com/github/rbolekr/ipynb/blob/main/Gymi_3_Info_Generator_f%C3%BCr_mathematische_%C3%9Cbungen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projekt Beschreibung

Mein Projekt ist ein Mathe-Übungen Generator. Es geht um den gleichen Typ von Aufgaben zu erstellen, sodass man eine bestimmte Art üben darf. Das Projekt setzt eigentlich nur Zahlen in einer schon bestimmten Vorlage ein, und kann danach den richtigen Antwort geben, und deine Lösung vergleichen und dir sagen, ob es richtig oder falsch ist.

## Technische Details

Dieses Projekt nutzt diese Moduls:
- [SymPy](https://www.sympy.org/en/index.html) für die symbolische Manipulation der verschiedene Funktionen, welcher das Programm erstellt.
- [Jupyter Widgets](https://ipywidgets.readthedocs.io/en/latest/index.html) für die Benutzerschnittstelle.

In [ ]:
import ipywidgets as widgets
from IPython.display import display

class Interface:
    def __init__(self, question_label_value='', answer_label_value='', check_button_click_handler=None):
        self.question_label_value = question_label_value
        self.answer_label_value = answer_label_value
        self.check_button_click_handler = check_button_click_handler

        self.tab = None
        self.answer_text = None
        self.check_button = None

        self.create_interface()

    def create_interface(self):
        # The interface will be composed of two tabs.
        self.tab = widgets.Tab()        

        # Tab 1: 'Question'
        question_label = widgets.HTML(
            value=f'<br><h1><center>{self.question_label_value}</center></h1><br><br>',
            placeholder='',
            description='',
            layout=widgets.Layout(width='75%')
        )

        self.answer_text = widgets.Text(
            value='',
            description='Antwort:',
            layout=widgets.Layout(width='75%')
        )

        # This is a nice-to-have: Initially the 'Check' button is disable and it's only enabled when the
        # contents of the 'Answer' text are not empty. To handle that we need to observe for change events.
        self.answer_text.observe(self.on_text_change, names='value')

        self.check_button = widgets.Button(
            description='Überprüfen',
            button_style='primary',
            layout=widgets.Layout(width='75%'),
            disabled=True
        )

        # Important: By default the handler will be notified of which button was clicked which is not useful.
        # We want the text of a different UI element so we have to intercept the call and provide it ourselves.
        self.check_button.on_click(self.check_button_click_handler)

        question_tab_box = widgets.VBox(
            [question_label, self.answer_text, self.check_button],
            layout=widgets.Layout(width='66%')
        )

        # Tab 2: 'Answer'
        answer_label = widgets.HTML(
            value=f'<br><h1><center>Die richtige Antwort ist: {self.answer_label_value}</center></h1><br><br>',
            placeholder='',
            description='',
            layout=widgets.Layout(width='75%')
        )

        answer_tab_box = widgets.VBox(
            [answer_label],
            layout=widgets.Layout(width='66%')
        )

        self.tab.children = [question_tab_box, answer_tab_box]
        self.tab.set_title(0, 'Frage')
        self.tab.set_title(1, 'Antwort')

    def handle_button_click(self, handler):
      return lambda b: handler(self.answer_text.value)

    def on_text_change(self, change):
        self.check_button.disabled = len(change['new'].strip()) == 0

    def display(self):
        display(self.tab)

# Generator 1: Multiplikation von 2 Zahlen im Bereich vom 1-10.

Dieses Generator könnte meinem 5-jährige Bruder in ein paar Jahren hilfreich sein.

In [ ]:
import random
from sympy import *

# Variables and function (simple multiplication) definition:
a, b = symbols('a, b')
f = Mul(a, b)

# Two random concrete values:
a_concrete = random.randint(1, 10)
b_concrete = random.randint(1, 10)
solution = int(f.evalf(subs={a:a_concrete, b:b_concrete}))

def click_handler(button):
    value = interface.answer_text.value.strip()
    try:
        answer_int_value = int(value)
        if answer_int_value == solution:
            print(f'Richtig! Bitte nochmals klicken für eine neue Frage.')
        else:
            print(f'{answer_int_value} ist falsch. Bitte nochmals probieren oder die Lösung anschauen.')
    except ValueError:
        print(f'"{value}" sieht wie kein Zahl aus. Bitte nochmals probieren oder die Lösung anschauen.')

interface = Interface(
    question_label_value=f'Was ist {a_concrete} x {b_concrete}?',
    answer_label_value=f'{solution}',
    check_button_click_handler=click_handler
)

interface.display()

# Generator 2: Polynomen ausmultiplizieren

Dieses Generator wird mehr für Ugymmi-Schüler*innen nützlich sein.

## Antworten überprüfen

Verschiedene gleichwertige Antworten sind schwierig für Sympy zu bewerten. Die Lösung für die Vergleichungen der Antworten in meinem Code war von dieser Quellen genommen:
- https://github.com/sympy/sympy/wiki/Faq#why-does-sympy-say-that-two-equal-expressions-are-unequal

## Arten von Fragen

Dieses Generator kann 3 Arten von Fragen erstellen:
1. Einzige Term (20% der Zeit). Ex: $4 x^{2} \left(x^{2} - 6 x + 2\right)$
2. binomische Ausmultiplizierung (40%). Ex: $(x - 4)(x + 2)$
3. binomische Ausmultiplizierung mit 2 Variablen (40%). Ex: $(3 x + 7 y)(x - 2 y)$

## Wie man Antworten eintragen soll

Um Antworten einzugeben, muss der Benutzer sie in Computer Notation schreiben
Ex:
$(3x + 7y)(x - 2y)$  = $3 x{^2} + x y - 14 y{2}$ aber was der Benutzer schreibt ist:
``
3*x**2+x*y-14*y**2"
``
( * zählt als Multiplikation, ** zählt als Exponent )

In [ ]:
import random
from sympy import *

categories = ['alpha', 'beta', 'gamma']
selected_category = random.choices(categories, weights=[0.2, 0.4, 0.4])[0]
x, y = symbols('x, y')
f = None

if selected_category == 'alpha':
    aa = random.randint(1, 10)
    bb = random.randint(1, 5)
    cc = random.randint(1, 10)
    dd = random.randint(2, 4)
    ee = random.randint(1, 10)
    ff = random.randint(1, 10)

    sign_aa = random.choice(['', '-'])
    sign_cc = random.choice(['', '-'])
    sign_dd = random.choice(['+', '-'])
    sign_ee = random.choice(['+', '-'])

    f = parse_expr(f"{sign_aa}{aa}*x**{bb}*({sign_cc}{cc}*x**{dd}{sign_dd}{ee}*x{sign_ee}{ff})")
    
elif selected_category == 'beta':
    aa = random.randint(1, 10)
    bb = random.randint(1, 10)
    cc = random.randint(1, 10)
    dd = random.randint(1, 10)

    sign_aa = random.choice(['', '-'])
    sign_bb = random.choice(['+', '-'])
    sign_cc = random.choice(['', '-'])
    sign_dd = random.choice(['+', '-'])
    
    f = parse_expr(f"({sign_aa}{aa}*x{sign_bb}{bb})*({sign_cc}{cc}*x{sign_dd}{dd})")

else:
    aa = random.randint(1, 10)
    bb = random.randint(1, 10)
    cc = random.randint(1, 10)
    dd = random.randint(1, 10)

    sign_aa = random.choice(['', '-'])
    sign_bb = random.choice(['+', '-'])
    sign_cc = random.choice(['', '-'])
    sign_dd = random.choice(['+', '-'])

    f = parse_expr(f"({sign_aa}{aa}*x{sign_bb}{bb}*y)*({sign_cc}{cc}*x{sign_dd}{dd}*y)")

solution = f.expand()

def click_handler(button):
    value = interface.answer_text.value.strip()
    
    try:
        answer = parse_expr(value)
        
        if answer.expand() == solution.expand():
            print(f'Richtig! Bitte nochmals klicken für eine neue Frage.')
        elif simplify(solution - answer) == 0:
            print(f'Richtig! Bitte nochmals klicken für eine neue Frage.')
        else:
            print(f'{value} ist falsch. Bitte nochmals probieren oder die Lösung anschauen.')
    except SyntaxError:
        print(f'"{value}" sieht wie kein Zahl aus. Achten Sie darauf, dass Sie das Multiplikationszeichen (*) rightig eingeben. Bitte nochmals probieren oder die Lösung anschauen. ')
    
interface = Interface(
    question_label_value=f'Bitte ausmultiplizieren :  {f}?',
    answer_label_value=f'{solution}',
    check_button_click_handler=click_handler
)

interface.display()

# Generator 3: Zahlen addieren und substrahieren



In [ ]:
import random
from sympy import *

# Variables and function (simple multiplication) definition:
a, b = symbols('a, b')
f = Add(a, b)

# Two random concrete values:
a_concrete = random.randint(-100, 100)
b_concrete = random.randint(-100, 100)
solution = int(f.evalf(subs={a:a_concrete, b:b_concrete}))

def click_handler(button):
    value = interface.answer_text.value.strip()
    try:
        answer_int_value = int(value)
        if answer_int_value == solution:
            print(f'Richtig! Bitte nochmals klicken für eine neue Frage.')
        else:
            print(f'{answer_int_value} ist falsch. Bitte nochmals probieren oder die Lösung anschauen.')
    except ValueError:
        print(f'"{value}" sieht wie kein Zahl aus. Bitte nochmals probieren oder die Lösung anschauen.')

interface = Interface(
    question_label_value=f'Was ist ({a_concrete}) + ({b_concrete})?',
    answer_label_value=f'{solution}',
    check_button_click_handler=click_handler
)

interface.display()

# Generator 4: Zahlen dividieren

In [ ]:
import random

# Two random concrete values:
a_concrete = random.randint(1, 20)
b_concrete = random.randint(1, 20)
c_concrete = a_concrete * b_concrete

solution = a_concrete

def click_handler(button):
    value = interface.answer_text.value.strip()
    try:
        answer_int_value = int(value)
        if answer_int_value == solution:
            print(f'Richtig! Bitte nochmals klicken für eine neue Frage.')
        else:
            print(f'{answer_int_value} ist falsch. Bitte nochmals probieren oder die Lösung anschauen.')
    except ValueError:
        print(f'"{value}" sieht wie kein Zahl aus. Bitte nochmals probieren oder die Lösung anschauen.')

interface = Interface(
    question_label_value=f'Was ist {c_concrete} / {b_concrete}?',
    answer_label_value=f'{solution}',
    check_button_click_handler=click_handler
)

interface.display()